# Business Loan Assistant Chatbot (GenAI Project)

## Project Overview
This project builds a Business Loan Assistant chatbot using a free LLM hosted on Hugging Face and Agentic AI. The chatbot is designed to help small business owners understand their loan eligibility, loan types, application requirements, and repayment terms.

In [10]:
from huggingface_hub import InferenceClient
from dotenv import load_dotenv
import os

load_dotenv()

client = InferenceClient(
    model="HuggingFaceH4/zephyr-7b-beta",
    token=os.getenv('HF_API_KEY')
)

#Agentic AI function to suggest loan types based on user input
def agentic_suggestion(user_message):
    user_message_lower = user_message.lower()

    if "income" in user_message_lower:
        import re
        income_match = re.search(r"\$?(\d{2,6})", user_message_lower)
        if income_match:
            income = int(income_match.group(1))
            if income < 50000:
                return (
                    "**Suggestion:**\n"
                    "- Based on your income level, you may qualify for a **Microloan**.\n"
                    "- Microloans are ideal for small or new businesses.\n"
                    "- They usually offer smaller amounts with flexible repayment terms."
                )
            else:
                return (
                    "**Suggestion:**\n"
                    "- With your income level, you might qualify for a **Growth Loan**.\n"
                    "- Growth Loans are suited for expanding established businesses."
                )
    
    if "started" in user_message_lower or "new business" in user_message_lower:
        return (
            "**Suggestion:**\n"
            "- Since your business is new, a **Microloan** might be the best early funding option."
        )

    if "operating for" in user_message_lower or "years" in user_message_lower:
        return (
            "**Suggestion:**\n"
            "- Since your business has been operating for several years, a **Growth Loan** could support your expansion."
        )

    return None  


def chat_with_loan_assistant(user_message):
    agent_response = agentic_suggestion(user_message)

    if agent_response:
        return agent_response #if we have an agentic response we can skip LLM
    
  
    response = client.text_generation(
    prompt=f"User: {user_message}\nAssistant (please format your answer with bullet points if listing options, limit your answer to 400 tokens, and bold important terms like Microloan or Growth Loan):",
    max_new_tokens=400, #changed from 200 to 400 to allow more words since answers were getting cut
    temperature=0.7,
    stop_sequences=["User:"]
)

    return response

In [6]:
#prompt engineering time! making sure its clear and clearly defines what is expected of AI
#made sure limit was 400 tokens
#modified prompts to make answers more precise.
system_prompt = """
You are a Business Loan Assistant helping small business owners understand their loan options.

Your job is to:
- Guide users based on their income, business age, and funding purpose.
- Suggest Microloans for new or low-income businesses.
- Suggest Growth Loans for established businesses needing expansion.
- Explain required documents: business plan, financial statements, and credit reports.
- Provide realistic, helpful answers without overpromising.

Be clear, supportive, follow the rules and be professional. If unsure, recommend contacting a BDC advisor.
"""

In [12]:
#testing the hugging face chatbot agentic ai (must return fast)
user_question = "I just started my business and make about $40,000 a year. What loans could I qualify for?"
response = chat_with_loan_assistant(user_question)

print("User:", user_question)
print("Assistant:", response)

User: I just started my business and make about $40,000 a year. What loans could I qualify for?
Assistant: **Suggestion:**
- Since your business is new, a **Microloan** might be the best early funding option.


In [11]:
user_question = "What documents do I need to apply for a business loan?"
response = chat_with_loan_assistant(user_question)

print("User:", user_question)
print("Assistant:", response)

User: What documents do I need to apply for a business loan?
Assistant: 

- Business plan: A comprehensive document that outlines your business goals, strategies, and financial projections. It should also include a marketing plan, management team biographies, and competitive analysis.

- Personal financial statement: A summary of your personal finances, including your income, expenses, and assets. Lenders want to see that you have a solid financial foundation and can manage the debt.

- Business financial statements: Profit and loss statements, balance sheets, and cash flow statements for at least the past three years. Lenders will want to see that your business is financially stable and has a history of profitability.

- Collateral: Depending on the type of loan, you may need to provide collateral, such as real estate, equipment, or inventory. This helps protect the lender in case you default on the loan.

- Legal documents: If you have any legal documents, such as articles of incorpo

In [13]:
user_question = "My business makes $75,000 per year and is 5 years old. What would you recommend?"
response = chat_with_loan_assistant(user_question)

print("User:", user_question)
print("Assistant:", response)

User: My business makes $75,000 per year and is 5 years old. What would you recommend?
Assistant: **Suggestion:**
- Since your business has been operating for several years, a **Growth Loan** could support your expansion.


In [17]:
user_question = "Can you give me 5 instances where a Growth Loan is better than a Microloan?"
response = chat_with_loan_assistant(user_question)

print("User:", user_question)
print("Assistant:", response)

User: Can you give me 5 instances where a Growth Loan is better than a Microloan?
Assistant: 
- Higher Loan Amount: Growth Loans typically offer much higher loan amounts than Microloans, allowing businesses to access the capital they need to finance larger expansion projects. While Microloans may cap at $50,000, Growth Loans can provide up to $1 million or more.
- Longer Repayment Terms: Growth Loans have longer repayment terms than Microloans, which can help businesses manage cash flow more effectively. Microloans typically require repayment within 1-5 years, while Growth Loans can offer repayment terms of 5-10 years or more.
- Collateral Requirements: While Microloans may require collateral, Growth Loans often accept alternative forms of collateral, such as intellectual property or future revenue streams, making it easier for small businesses to secure financing.
- Customized Loan Structures: Growth Loans can be structured to meet the unique needs of each business, with flexible repa

In [14]:
user_question = "Can I apply for a loan if I don't have a business plan ready yet?"
response = chat_with_loan_assistant(user_question)

print("User:", user_question)
print("Assistant:", response)

User: Can I apply for a loan if I don't have a business plan ready yet?
Assistant: 

1. Microloan: In some cases, lenders may provide microloans without a formal business plan. These loans are generally smaller in amount and are intended to help borrowers establish or grow a small business. However, lenders may still require that you provide a detailed overview of your business idea, including your target market, competitive landscape, and financial projections.

2. Growth Loan: For larger loans, lenders typically require a formal business plan as part of the application process. This is because a detailed plan helps the lender to assess the viability of your business and the likelihood of repayment. If you are seeking a growth loan, it may be beneficial to work with a business advisor or mentor to develop a solid business plan before applying.

3. Alternative Financing: Some alternative financing options, such as crowdfunding or peer-to-peer lending platforms, may be less stringent in

#### Observations

- The chatbot was able to recommend Microloans or Growth Loans correctly for most new businesses.
- Some longer answers were cut off (can be improved by adjusting max_new_tokens).
- Responses were professional and realistic.
- Sometimes the bot could suggest contacting an advisor more proactively.

#### Strengths

- Free model used effectively.
- Business relevance maintained across tests.
- Adaptable to different client profiles.

#### Limitations

- Since it is a small free model, it has token limitations.
- Hugging Face models can be slightly slower than OpenAI APIs.


## Conclusion

In this project I successfully demonstrates a GenAI-powered Business Loan Assistant using a free Hugging Face LLM.
The assistant is capable of answering a wide range of business loan queries, supporting decision-making for small business owners as well as give responses based on what it knows about the user.
This project highlights the potential of integrating GenAI into real-world financial services.
